In [1]:
from imputationLibrary import decompose, forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, randomSampleImputation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn import preprocessing
from datetime import datetime, timedelta
from tsmoothie.smoother import *

In [2]:
INPUT = 'output/\CompleteWeeklyIndexes'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df_train = df.loc['2000-01-01':'2015-01-01'] #Selecting training data
df_filled = df_train.fillna(0)
df_filled_test = df_test.fillna(0)

In [4]:
df_decomposed_resid = pd.DataFrame()
df_decomposed_trend = pd.DataFrame()
df_decomposed_seasonal = pd.DataFrame()

df_decomposed_resid_test = pd.DataFrame()
df_decomposed_trend_test = pd.DataFrame()
df_decomposed_seasonal_test = pd.DataFrame()

df_nan = df_train.isna()
df_nan_test = df_test.isna()

#df_mask_nan_resid = pd.DataFrame()
#for col in df_nan.columns:
#    df_mask_nan[col+"_resid"] = df_nan.loc[:,col]
#    df_mask_nan[col+"_trend"] = df_nan.loc[:,col]
#    df_mask_nan[col+"_seasonal"] = df_nan.loc[:,col]

In [5]:
def plot_ac(df, name):
    normalized = preprocessing.scale([np.array(df.fillna(0))])
    corr = signal.correlate(normalized, normalized, mode='full')
    plt.plot(corr[0], 'o-', markersize=2)
    plt.axhline(2/np.sqrt(len(df)), ls=':')
    plt.axhline(-2/np.sqrt(len(df)), ls=':')
    plt.title(name + ' auto-correlation')
    plt.show()

In [6]:
def decompose_and_plot(ts, flag_plot = False):
    ts_decomposed = decompose.decompose(ts)
    if flag_plot:
        ts_decomposed.plot()
        plt.show()
        print("Resid is white noise? ", nature.isWhiteNoise(ts_decomposed.resid))
        print("Resid is seasonal noise? ", nature.isSeasonal(ts_decomposed.resid))
        print("Resid is trended noise? ", nature.isTrended(ts_decomposed.resid))
        print("Resid is seasonal and trended noise? ", nature.isTrendedAndSeasonal(ts_decomposed.resid))
        plot_ac(ts_decomposed.resid, "Resid")
        plot_ac(ts_decomposed.trend, "Trend")
        plot_ac(ts_decomposed.seasonal, "Seasonal")
    return ts_decomposed.resid, ts_decomposed.trend, ts_decomposed.seasonal
    

In [7]:
for col in df_filled.columns:
    resid, trend, seasonal = decompose_and_plot(df_filled.loc[:,col])
    df_decomposed_resid[col] = resid
    df_decomposed_trend[col] = trend
    df_decomposed_seasonal[col] = seasonal

In [8]:
for col in df_filled_test.columns:
    resid, trend, seasonal = decompose_and_plot(df_filled_test.loc[:,col])
    df_decomposed_resid_test[col] = resid
    df_decomposed_trend_test[col] = trend
    df_decomposed_seasonal_test[col] = seasonal

In [9]:
df_final_resid = df_decomposed_resid.mask(df_nan == True, np.nan)
df_final_trend = df_decomposed_trend.mask(df_nan == True, np.nan)
df_final_seasonal = df_decomposed_seasonal.mask(df_nan == True, np.nan)

df_final_resid_test = df_decomposed_resid_test.mask(df_nan_test == True, np.nan)
df_final_trend_test = df_decomposed_trend_test.mask(df_nan_test == True, np.nan)
df_final_seasonal_test = df_decomposed_seasonal_test.mask(df_nan_test == True, np.nan)

In [10]:
df_white_noise_train, df_white_noise_test =  meanImputation.input(df_final_resid, df_final_resid_test)

In [11]:
df_seasonal_train, df_seasonal_test = splineInterpolation.input(df_final_seasonal, df_final_seasonal_test)

In [12]:
df_trended_train, df_trended_test = forwardFilling.input(df_final_trend, df_final_trend_test)

In [13]:
df_final_train = df_white_noise_train + df_seasonal_train + df_trended_train
df_final_test = df_white_noise_test + df_seasonal_test + df_trended_test

In [14]:
df_final_train

sp500_Close  sp500_Volume  vix_Close  vix_Volume     dji_Close  \
2000-01-07  1431.932646  3.031014e+08  22.570398         0.0  10816.675933   
2000-01-14  1435.148573  3.024809e+08  22.639629         0.0  10827.929670   
2000-01-21  1440.525857  1.029892e+09  22.065281         0.0  10839.458979   
2000-01-28  1433.677528  1.164465e+09  22.230324         0.0  10748.429743   
2000-02-04  1426.321809  1.350010e+09  23.291117         0.0  10706.555429   
2000-02-11  1426.205604  1.204788e+09  23.003351         0.0  10733.604949   
2000-02-18  1422.236451  1.147484e+09  23.055603         0.0  10712.265940   
2000-02-25  1420.085998  9.953825e+08  23.186741         0.0  10713.372821   
2000-03-03  1424.362131  8.268907e+08  22.811845         0.0  10735.154622   
2000-03-10  1423.807927  1.029769e+09  22.987135         0.0  10725.354799   
2000-03-17  1408.518647  1.087203e+09  23.993302         0.0  10598.735975   
2000-03-24  1413.352445  1.261312e+09  23.669308         0.0  10628.459118   
2000-03-31  1418.415304  1.358513e+09  23.033563         0.0  10676.561743   
2000-04-07  1424.756013  1.110069e+09  22.719894         0.0  10732.452788   
2000-04-14  1432.447597  9.381146e+08  21.952583         0.0  10805.749436   
2000-04-21  1439.258833  9.631342e+08  21.020049         0.0  10852.924419   
2000-04-28  1437.965686  1.212580e+09  21.093009         0.0  10842.738672   
2000-05-05  1441.421894  1.256118e+09  21.190662         0.0  10901.441960   
2000-05-12  1448.818328  1.107149e+09  21.006045         0.0  10978.011573   
2000-05-19  1454.159357  1.376339e+09  21.270678         0.0  11038.584555   
2000-05-26  1450.795856  1.072094e+09  21.264133         0.0  10996.516166   
2000-06-02  1444.774948  1.148551e+09  21.954105         0.0  10933.593566   
2000-06-09  1444.645377  9.481215e+08  21.876320         0.0  10907.683859   
2000-06-16  1441.325927  1.065632e+09  22.431835         0.0  10868.885693   
2000-06-23  1433.008167  9.979968e+08  22.711224         0.0  10809.241240   
2000-06-30  1430.715785  9.584805e+08  22.183753         0.0  10776.776133   
2000-07-07  1462.835022  8.375500e+08  20.287500         0.0  10540.429932   
2000-07-14  1491.047998  9.615600e+08  19.964001         0.0  10751.798047   
2000-07-21  1492.389966  9.513200e+08  19.346001         0.0  10763.539844   
2000-07-28  1452.138013  1.044380e+09  19.884000         0.0  10599.774024   
...                 ...           ...        ...         ...           ...   
2014-06-06  1933.397974  2.829538e+09  11.588000         0.0  16792.777734   
2014-06-13  1942.444019  2.772774e+09  11.696000         0.0  16848.566015   
2014-06-20  1951.819995  3.250200e+09  11.358000         0.0  16872.932031   
2014-06-27  1958.059985  3.196694e+09  11.518000         0.0  16864.174219   
2014-07-04  1922.413286  3.231657e+09  13.105528         0.0  16674.897065   
2014-07-11  1916.042458  3.432390e+09  13.427954         0.0  16665.863753   
2014-07-18  1928.322562  3.421547e+09  12.420032         0.0  16787.228864   
2014-07-25  1924.816041  3.389244e+09  13.628608         0.0  16758.153431   
2014-08-01  1926.896835  3.461281e+09  13.229634         0.0  16788.733427   
2014-08-08  1924.334878  3.161739e+09  13.924971         0.0  16747.972367   
2014-08-15  1922.234841  2.902519e+09  13.587636         0.0  16707.593168   
2014-08-22  1916.145462  3.086696e+09  14.797561         0.0  16657.749813   
2014-08-29  1916.271600  3.094913e+09  14.893286         0.0  16644.491727   
2014-09-05  1909.443753  3.416799e+09  15.763102         0.0  16547.911429   
2014-09-12  1912.673208  3.750549e+09  15.505345         0.0  16558.432589   
2014-09-19  1917.887626  3.412573e+09  15.265480         0.0  16618.855457   
2014-09-26  1908.430907  3.417837e+09  17.096252         0.0  16563.490933   
2014-10-03  1901.382689  3.440720e+09  17.869851         0.0  16479.294331   
2014-10-10  1907.132417  3.478250e+09  17.038507         0.0  16538.757942   
2014-10-17  1912.927545  3.508093e

In [15]:
### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

smoother_train = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_train.smooth(df_final_train.T)

smoother_test = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_test.smooth(df_final_test.T)

<tsmoothie.smoother.KalmanSmoother>

In [16]:
df_filled = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
df_filled_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)

In [17]:
df_filled

sp500_Close  sp500_Volume  vix_Close  vix_Volume     dji_Close  \
2000-01-07  1421.900770  6.288783e+08  22.297591         0.0  10713.504249   
2000-01-14  1426.197628  7.021191e+08  22.399208         0.0  10741.746309   
2000-01-21  1428.583145  8.552925e+08  22.452107         0.0  10752.037430   
2000-01-28  1428.459039  9.734933e+08  22.581740         0.0  10744.136054   
2000-02-04  1427.171181  1.053396e+09  22.781004         0.0  10734.680544   
2000-02-11  1425.934673  1.073842e+09  22.877550         0.0  10730.168772   
2000-02-18  1424.527051  1.067952e+09  22.948228         0.0  10724.301220   
2000-02-25  1423.462219  1.046016e+09  22.996725         0.0  10720.184698   
2000-03-03  1422.958748  1.034074e+09  23.006527         0.0  10716.786449   
2000-03-10  1422.061983  1.063443e+09  23.054596         0.0  10709.082238   
2000-03-17  1420.704764  1.099411e+09  23.115503         0.0  10697.504342   
2000-03-24  1421.674832  1.137673e+09  23.000214         0.0  10705.074027   
2000-03-31  1424.199976  1.151044e+09  22.750553         0.0  10727.367073   
2000-04-07  1427.772460  1.122756e+09  22.443874         0.0  10759.221418   
2000-04-14  1431.837961  1.096852e+09  22.081744         0.0  10795.825172   
2000-04-21  1435.670361  1.102554e+09  21.745384         0.0  10829.833157   
2000-04-28  1438.673063  1.135996e+09  21.554202         0.0  10858.606050   
2000-05-05  1441.704649  1.153969e+09  21.455491         0.0  10889.931085   
2000-05-12  1444.679647  1.151354e+09  21.410054         0.0  10918.327838   
2000-05-19  1446.713283  1.157423e+09  21.445777         0.0  10934.158451   
2000-05-26  1447.143977  1.119555e+09  21.516894         0.0  10928.476570   
2000-06-02  1446.731054  1.091038e+09  21.638936         0.0  10908.569853   
2000-06-09  1446.596924  1.050896e+09  21.698300         0.0  10883.057933   
2000-06-16  1446.741432  1.031203e+09  21.722417         0.0  10852.039821   
2000-06-23  1447.857928  1.004530e+09  21.605032         0.0  10817.093193   
2000-06-30  1451.833286  9.790836e+08  21.266877         0.0  10783.180649   
2000-07-07  1459.919831  9.576903e+08  20.745990         0.0  10750.035537   
2000-07-14  1467.308407  9.602675e+08  20.317684         0.0  10758.317412   
2000-07-21  1469.831758  9.625311e+08  19.961216         0.0  10767.415838   
2000-07-28  1467.725606  9.669824e+08  19.729073         0.0  10776.805497   
...                 ...           ...        ...         ...           ...   
2014-06-06  1921.443843  3.008959e+09  12.059647         0.0  16719.540020   
2014-06-13  1928.828192  3.038227e+09  12.038568         0.0  16756.376214   
2014-06-20  1933.379891  3.120700e+09  12.088028         0.0  16774.020548   
2014-06-27  1934.132297  3.177355e+09  12.285496         0.0  16771.121612   
2014-07-04  1929.987787  3.230197e+09  12.638381         0.0  16748.852914   
2014-07-11  1927.248304  3.282777e+09  12.899619         0.0  16740.623969   
2014-07-18  1926.641395  3.305444e+09  13.056863         0.0  16746.599560   
2014-07-25  1925.590211  3.304888e+09  13.343065         0.0  16743.700965   
2014-08-01  1924.586328  3.287459e+09  13.573640         0.0  16737.164448   
2014-08-08  1923.013361  3.235272e+09  13.874393         0.0  16719.570022   
2014-08-15  1921.069834  3.197817e+09  14.166284         0.0  16695.558216   
2014-08-22  1918.787934  3.219461e+09  14.575026         0.0  16668.412529   
2014-08-29  1916.930120  3.267690e+09  14.940219         0.0  16642.683682   
2014-09-05  1915.100444  3.350483e+09  15.315597         0.0  16615.888464   
2014-09-12  1914.299250  3.419989e+09  15.602118         0.0  16601.993954   
2014-09-19  1913.720775  3.423333e+09  15.908512         0.0  16596.122605   
2014-09-26  1912.206716  3.428771e+09  16.343892         0.0  16585.018082   
2014-10-03  1911.346077  3.436336e+09  16.629016         0.0  16577.535448   
2014-10-10  1912.376563  3.442961e+09  16.666076         0.0  16589.018332   
2014-10-17  1914.353815  3.442464e

In [18]:
# Subsample para weekly

# Generating weekly sampled dataset
def generate_weekly(df, initial_friday):
    df_friday = initial_friday #datetime(2000,1,7)

    df_weekly = pd.DataFrame(columns = df.columns)
    
    while df_friday <= df.tail(1).index:
        df_weekly = df_weekly.append(df.loc[df_friday])
        df_friday+=timedelta(days=7)
    return df_weekly

In [19]:
df_train_weekly = generate_weekly(df_filled, datetime(2000,1,7))
df_test_weekly = generate_weekly(df_filled_test, datetime(2015,1,2))

In [20]:
df_train_weekly.to_csv(r'output\/CompleteIndexesWeeklyTrainDecompKalman.csv', index = True)
df_test_weekly.to_csv(r'output\/CompleteIndexesWeeklyTestDecompKalman.csv', index = True)

In [21]:
df_train_weekly.head(50)

sp500_Close  sp500_Volume  vix_Close  vix_Volume     dji_Close  \
2000-01-07  1421.900770  6.288783e+08  22.297591         0.0  10713.504249   
2000-01-14  1426.197628  7.021191e+08  22.399208         0.0  10741.746309   
2000-01-21  1428.583145  8.552925e+08  22.452107         0.0  10752.037430   
2000-01-28  1428.459039  9.734933e+08  22.581740         0.0  10744.136054   
2000-02-04  1427.171181  1.053396e+09  22.781004         0.0  10734.680544   
2000-02-11  1425.934673  1.073842e+09  22.877550         0.0  10730.168772   
2000-02-18  1424.527051  1.067952e+09  22.948228         0.0  10724.301220   
2000-02-25  1423.462219  1.046016e+09  22.996725         0.0  10720.184698   
2000-03-03  1422.958748  1.034074e+09  23.006527         0.0  10716.786449   
2000-03-10  1422.061983  1.063443e+09  23.054596         0.0  10709.082238   
2000-03-17  1420.704764  1.099411e+09  23.115503         0.0  10697.504342   
2000-03-24  1421.674832  1.137673e+09  23.000214         0.0  10705.074027   
2000-03-31  1424.199976  1.151044e+09  22.750553         0.0  10727.367073   
2000-04-07  1427.772460  1.122756e+09  22.443874         0.0  10759.221418   
2000-04-14  1431.837961  1.096852e+09  22.081744         0.0  10795.825172   
2000-04-21  1435.670361  1.102554e+09  21.745384         0.0  10829.833157   
2000-04-28  1438.673063  1.135996e+09  21.554202         0.0  10858.606050   
2000-05-05  1441.704649  1.153969e+09  21.455491         0.0  10889.931085   
2000-05-12  1444.679647  1.151354e+09  21.410054         0.0  10918.327838   
2000-05-19  1446.713283  1.157423e+09  21.445777         0.0  10934.158451   
2000-05-26  1447.143977  1.119555e+09  21.516894         0.0  10928.476570   
2000-06-02  1446.731054  1.091038e+09  21.638936         0.0  10908.569853   
2000-06-09  1446.596924  1.050896e+09  21.698300         0.0  10883.057933   
2000-06-16  1446.741432  1.031203e+09  21.722417         0.0  10852.039821   
2000-06-23  1447.857928  1.004530e+09  21.605032         0.0  10817.093193   
2000-06-30  1451.833286  9.790836e+08  21.266877         0.0  10783.180649   
2000-07-07  1459.919831  9.576903e+08  20.745990         0.0  10750.035537   
2000-07-14  1467.308407  9.602675e+08  20.317684         0.0  10758.317412   
2000-07-21  1469.831758  9.625311e+08  19.961216         0.0  10767.415838   
2000-07-28  1467.725606  9.669824e+08  19.729073         0.0  10776.805497   
2000-08-04  1468.620046  9.559009e+08  19.467368         0.0  10821.117977   
2000-08-11  1474.196795  9.384564e+08  19.104708         0.0  10897.526556   
2000-08-18  1479.811198  9.215728e+08  18.775137         0.0  10965.469053   
2000-08-25  1483.522293  9.148422e+08  18.666097         0.0  11011.817182   
2000-09-01  1483.082915  9.332900e+08  18.861449         0.0  11030.413921   
2000-09-08  1476.540804  9.714877e+08  19.411394         0.0  11013.543597   
2000-09-15  1465.372600  1.014932e+09  19.920831         0.0  10946.251892   
2000-09-22  1451.053354  1.051651e+09  20.732319         0.0  10842.689432   
2000-09-29  1436.698584  1.083332e+09  21.665828         0.0  10751.262478   
2000-10-06  1422.080544  1.104908e+09  22.733269         0.0  10667.965063   
2000-10-13  1406.095804  1.121653e+09  24.013269         0.0  10576.861934   
2000-10-20  1396.941882  1.132201e+09  24.753337         0.0  10531.472124   
2000-10-27  1392.999745  1.125727e+09  25.136365         0.0  10565.152365   
2000-11-03  1391.466470  1.107853e+09  25.317596         0.0  10633.076262   
2000-11-10  1383.978630  1.074454e+09  25.723184         0.0  10651.097043   
2000-11-17  1371.635411  1.064084e+09  26.039710         0.0  10628.063189   
2000-11-24  1358.989610  1.052796e+09  26.287956         0.0  10602.654189   
2000-12-01  1350.375041  1.078953e+09  26.404684         0.0  10606.139719   
2000-12-08  1345.491099  1.130500e+09  26.233618         0.0  10631.724317   
2000-12-15  1338.994200  1.172498e+09  26.284601         0.0  10645.179835   

              dji_Volume    ndx_C

In [22]:
df

sp500_Close  sp500_Volume  vix_Close  vix_Volume     dji_Close  \
2000-01-07  1420.333984  1.068760e+09  25.016000         0.0  11250.781836   
2000-01-14  1448.648023  1.033940e+09  21.684000         0.0  11587.958008   
2000-01-21  1449.492493  1.113750e+09  21.447500         0.0  11413.272461   
2000-01-28  1394.874023  1.106420e+09  23.960000         0.0  10967.587890   
2000-02-04  1412.439990  1.041000e+09  23.014001         0.0  10992.404102   
2000-02-11  1416.323975  1.020160e+09  22.886000         0.0  10726.277930   
2000-02-18  1382.802002  1.023060e+09  23.996000         0.0  10506.685938   
2000-02-25  1349.912506  1.063475e+09  24.832500         0.0  10121.330078   
2000-03-03  1376.918018  1.170760e+09  21.992000         0.0  10167.401953   
2000-03-10  1382.071972  1.161580e+09  22.616000         0.0   9952.522266   
2000-03-17  1411.569995  1.238060e+09  22.548000         0.0  10223.122070   
2000-03-24  1501.189990  1.038440e+09  22.344000         0.0  10937.372070   
2000-03-31  1505.321997  1.068560e+09  24.614000         0.0  10976.569922   
2000-04-07  1501.151978  1.109412e+09  26.220000         0.0  11129.287891   
2000-04-14  1453.858008  1.062540e+09  29.022000         0.0  10965.617773   
2000-04-21  1426.264984  1.052925e+09  26.985001         0.0  10717.234863   
2000-04-28  1457.128003  1.007000e+09  26.394000         0.0  10919.685937   
2000-05-05  1434.367993  9.401400e+08  28.862000         0.0  10602.676172   
2000-05-12  1409.626026  9.004800e+08  28.162000         0.0  10532.700000   
2000-05-19  1442.071997  8.584400e+08  24.516000         0.0  10783.244140   
2000-05-26  1386.634009  9.196600e+08  25.048000         0.0  10424.665820   
2000-06-02  1442.279999  9.818000e+08  22.777500         0.0  10624.104981   
2000-06-09  1463.089990  8.567200e+08  22.629999         0.0  10729.301953   
2000-06-16  1465.833984  9.803800e+08  21.330000         0.0  10607.624024   
2000-06-23  1466.947998  9.666200e+08  21.292000         0.0  10454.322070   
2000-06-30  1451.534009  1.119440e+09  20.756001         0.0  10484.233984   
2000-07-07  1462.835022  8.375500e+08  20.287500         0.0  10540.429932   
2000-07-14  1491.047998  9.615600e+08  19.964001         0.0  10751.798047   
2000-07-21  1492.389966  9.513200e+08  19.346001         0.0  10763.539844   
2000-07-28  1452.138013  1.044380e+09  19.884000         0.0  10599.774024   
...                 ...           ...        ...         ...           ...   
2019-08-30  2897.303955  3.134658e+09  19.168000         0.0  26095.671875   
2019-09-06  2949.690002  3.422258e+09  17.065000         0.0  26499.775391   
2019-09-13  2995.141944  3.926626e+09  14.608000         0.0  27056.789453   
2019-09-20  3001.850000  4.145610e+09  14.486000         0.0  27072.912110   
2019-09-27  2976.532080  3.338902e+09  16.242000         0.0  26887.967969   
2019-10-04  2933.447998  3.442528e+09  18.304000         0.0  26468.649610   
2019-10-11  2931.929981  3.164224e+09  17.986000         0.0  26460.265625   
2019-10-18  2987.133935  3.100116e+09  13.966000         0.0  26922.044141   
2019-10-25  3008.014014  3.450270e+09  13.766000         0.0  26842.655859   
2019-11-01  3045.509961  3.791334e+09  12.832000         0.0  27148.492188   
2019-11-08  3081.586035  4.146992e+09  12.670000         0.0  27560.668359   
2019-11-15  3097.995996  3.324508e+09  12.694000         0.0  27790.684375   
2019-11-22  3112.900000  3.601798e+09  12.714000         0.0  27886.647656   
2019-11-29  3142.192444  3.220808e+09  11.945000         0.0  28100.890137   
2019-12-06  3116.633984  3.490478e+09  14.762000         0.0  27725.695703   
2019-12-13  3149.495996  3.533976e+09  14.620000         0.0  27994.010547   
2019-12-20  3200.339990  4.415626e+09  12.404000         0.0  28314.876172   
2019-12-27  3231.829956  2.236625e+09  12.840000         0.0  28583.407227   
2020-01-03  3236.192566  3.206660e+09  13.772500         0.0  28626.065430   
2020-01-10  3255.312012  3.533340e